# Creamos un archivo con las provincias donde concetamos con API del Gobierno, para extraer los datos de "Latidud" y "Longitud".

De esta manera lo podremos usar como PK para nuestro proyecto en BI

In [1]:
import pandas as pd
import requests

In [11]:


# Cargar el archivo original
file_path = 'C:/Users/Pedro/Desktop/ingreso_percapita_argentina_localidades_reales_2010_2024.csv'
data = pd.read_csv(file_path)

# Obtener el listado completo de localidades y departamentos desde la API de Georef
def obtener_localidades_completas():
    url = 'https://apis.datos.gob.ar/georef/api/localidades?campos=id,nombre,provincia.nombre,centroide.lat,centroide.lon&max=5000'
    response = requests.get(url)
    if response.status_code == 200:
        datos = response.json()
        localidades = datos.get('localidades', [])
        return localidades
    return []

# Convertir el listado de localidades en un DataFrame
localidades_completas = obtener_localidades_completas()
df_localidades_completas = pd.DataFrame(localidades_completas)

# Separar las coordenadas del centroide en latitud y longitud
df_localidades_completas['Latitud'] = df_localidades_completas['centroide'].apply(lambda x: x['lat'] if pd.notnull(x) else None)
df_localidades_completas['Longitud'] = df_localidades_completas['centroide'].apply(lambda x: x['lon'] if pd.notnull(x) else None)

# Renombrar columnas para que coincidan con el archivo original
df_localidades_completas.rename(columns={
    'nombre': 'Localidad/Departamento',
    'provincia': 'Provincia'
}, inplace=True)

# Eliminar la columna 'centroide' ya que hemos separado latitud y longitud
df_localidades_completas.drop(columns=['centroide'], inplace=True)

# Verificar si las columnas renombradas existen en el DataFrame
print(df_localidades_completas.columns)

# Identificar las localidades/departamentos faltantes
df_faltantes = df_localidades_completas[~df_localidades_completas[['Provincia', 'Localidad/Departamento']].apply(tuple, axis=1).isin(data[['Provincia', 'Localidad/Departamento']].apply(tuple, axis=1))]

# Agregar las localidades/departamentos faltantes al archivo original
data_enriquecido = pd.concat([data, df_faltantes], ignore_index=True)

# Guardar el nuevo archivo
output_path = 'C:/Users/Pedro/Desktop/ingreso_percapita_argentina_localidades_reales_2010_2024_enriquecido.csv'
data_enriquecido.to_csv(output_path, index=False)

print(f"El archivo enriquecido ha sido guardado en {output_path}")

Index(['id', 'Localidad/Departamento', 'Provincia', 'Latitud', 'Longitud'], dtype='object')
El archivo enriquecido ha sido guardado en C:/Users/Pedro/Desktop/ingreso_percapita_argentina_localidades_reales_2010_2024_enriquecido.csv


In [4]:

print(df_localidades_completas.columns.tolist())


['centroide', 'id', 'Localidad/Departamento', 'provincia']
